In [3]:
import pandas as pd
import pandas_utils as pu
import rdkit_mol_identifiers as rdkit
from rdkit.Chem import PandasTools

## Read multiple library files with inchikey column
only inchikey needed

In [24]:
libraries = {
  "mce": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\mce_library_all_cleaned.tsv",
  "nih": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\nih_library_new_headers_cleaned.tsv",
  "warth_xeno_poly": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Warth_Vienna_Xenobiotics _PoPhe_new_headers_cleaned.tsv",
  "petras_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Petras_Tübingen_Natural_Product_Library_cleaned.tsv",
  "koellensperger_mix": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Koellensperger_compounds_cleaned.tsv",
  "targetmol_np_4320": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\L6000-Targetmol-Natural Product Library for HTS-4320cpds_script_cleaned.tsv",
  "targetmol_np_3720": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\L6010-Targetmol-Natural Product Library-3720cpds_cleaned.tsv",
  "selleckchem_subset": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem_subset_L5000-1w_cleaned.tsv",
  "selleckchem_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1400-Natural-Product-Library-96-well_new_headers_cleaned.tsv",
  "selleckchem_phenol_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1410-Natural Phenol Compound Library-96-well_new_headers_cleaned.tsv",
  "selleckchem_terpenoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1420-Natural Terpenoid Compound Library-96-well_new_headers_cleaned.tsv",
  "selleckchem_organic_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7600-Natural-Organic-Compound-Library-96-well_new_headers_cleaned.tsv",
  "selleckchem_flavonoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7700-Flavonoid-Compound-Library-96-well_new_headers_cleaned.tsv",
  "selleckchem_alkaloid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7900-Alkaloid-Compound-Library-96-well_new_headers_cleaned.tsv",
  "nencka_mce": r"C:\git\msn_library\data\compound_libraries\Radim_mce_complete_cleaned.tsv",
  "puretitre": r"C:\git\msn_library\data\compound_libraries\raw_data\puretitre_caithnessbiotechnologies_new_header_cleaned.tsv",
  "analyticon_megx_617": r"C:\git\msn_library\data\compound_libraries\raw_data\MEGx_Release_2023_09_01\Analyticon_MEGx_Release_2023_09_01_617_cpds_cleaned.tsv",
  "analyticon_natx_387": r"C:\git\msn_library\data\compound_libraries\raw_data\NATx_Release_2023_09_01\Analyticon_NATx_Release_2023_09_01_Newly_available_387_cpds_cleaned.tsv"
}

acquired = [
  "mce", 
  "nih",
  "nencka_mce"
]

collaborators = [
  "petras_np_library",
  "warth_xeno_poly",
  "koellensperger_mix",
]

In [25]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["inchikey"]].copy()
  df = df.dropna(subset="inchikey").drop_duplicates(["inchikey"]).set_index(["inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)
  
  
merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]


merged_df

C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (139) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (7,51,55,57,58,60,61,62,64,65,67,69,75,76,82,87,90,99,157) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (71,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (44,51,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (39,77,122,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,selleckchem_np_library,selleckchem_phenol_library,selleckchem_terpenoid_library,selleckchem_organic_library,selleckchem_flavonoid_library,selleckchem_alkaloid_library,nencka_mce,puretitre,analyticon_megx_617,analyticon_natx_387,entries,split_inchikey
inchikey,,,,,,,,,,,,,,,,,,,,
BGVLELSCIHASRV-QPEQYQDCSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BGVLELSCIHASRV
SWMDAPWAQQTBOG-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,SWMDAPWAQQTBOG
PZUSGRHVYDQLHR-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,PZUSGRHVYDQLHR
FTSDLONCFCQDGA-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FTSDLONCFCQDGA
KPFZCKDPBMGECB-WGDLNXRISA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,KPFZCKDPBMGECB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FMDOZEYCUQMGGS-FTUNRGFVSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,FMDOZEYCUQMGGS
PAIDVHBPKNSPRV-DACLVMHWSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,PAIDVHBPKNSPRV
YUGRMJLFPJDYRP-JSJNYSNDSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,YUGRMJLFPJDYRP


In [26]:
merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)
merged_df

,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,selleckchem_np_library,selleckchem_phenol_library,...,selleckchem_alkaloid_library,nencka_mce,puretitre,analyticon_megx_617,analyticon_natx_387,entries,split_inchikey,acquired,collaborators,accessible
inchikey,,,,,,,,,,,,,,,,,,,,,
BGVLELSCIHASRV-QPEQYQDCSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,BGVLELSCIHASRV,True,False,True
SWMDAPWAQQTBOG-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,SWMDAPWAQQTBOG,True,False,True
PZUSGRHVYDQLHR-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,PZUSGRHVYDQLHR,True,False,True
FTSDLONCFCQDGA-UHFFFAOYSA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,FTSDLONCFCQDGA,True,False,True
KPFZCKDPBMGECB-WGDLNXRISA-N,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,KPFZCKDPBMGECB,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FMDOZEYCUQMGGS-FTUNRGFVSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,FMDOZEYCUQMGGS,False,False,False
PAIDVHBPKNSPRV-DACLVMHWSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,PAIDVHBPKNSPRV,False,False,False
YUGRMJLFPJDYRP-JSJNYSNDSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,YUGRMJLFPJDYRP,False,False,False


In [27]:
not_acquired = merged_df.loc[merged_df["acquired"] == False]
not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [28]:
not_acquired

,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,selleckchem_np_library,selleckchem_phenol_library,...,selleckchem_alkaloid_library,nencka_mce,puretitre,analyticon_megx_617,analyticon_natx_387,entries,split_inchikey,acquired,collaborators,accessible
inchikey,,,,,,,,,,,,,,,,,,,,,
NRCXNPKDOMYPPJ-HYORBCNSSA-N,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NRCXNPKDOMYPPJ,False,True,True
LINOMUASTDIRTM-QGRHZQQGSA-N,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,LINOMUASTDIRTM,False,True,True
UKOTXHQERFPCBU-XBXCNEFVSA-N,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,UKOTXHQERFPCBU,False,True,True
ZACLXWTWERGCLX-MDUHGFIHSA-N,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,ZACLXWTWERGCLX,False,True,True
GYNOTJLCULOEIM-XKRJZGAWSA-N,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,GYNOTJLCULOEIM,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FMDOZEYCUQMGGS-FTUNRGFVSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,FMDOZEYCUQMGGS,False,False,False
PAIDVHBPKNSPRV-DACLVMHWSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,PAIDVHBPKNSPRV,False,False,False
YUGRMJLFPJDYRP-JSJNYSNDSA-N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,1,YUGRMJLFPJDYRP,False,False,False


In [29]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "nencka_mce", "puretitre", "analyticon_megx_617", "analyticon_natx_387"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


Column targetmol_np_4320 has 1773 True values.
Column targetmol_np_3720 has 1301 True values.
Column selleckchem_subset has 9994 True values.
Column selleckchem_organic_library has 267 True values.
Column selleckchem_terpenoid_library has 111 True values.
Column selleckchem_flavonoid_library has 37 True values.
Column selleckchem_alkaloid_library has 47 True values.
Column selleckchem_np_library has 510 True values.
Column nencka_mce has 0 True values.
Column puretitre has 22 True values.
Column analyticon_megx_617 has 545 True values.
Column analyticon_natx_387 has 387 True values.


In [30]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "nencka_mce", "puretitre", "analyticon_megx_617", "analyticon_natx_387"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")

Column targetmol_np_4320 has 1742 True values.
Column targetmol_np_3720 has 1273 True values.
Column selleckchem_subset has 9994 True values.
Column selleckchem_organic_library has 260 True values.
Column selleckchem_terpenoid_library has 111 True values.
Column selleckchem_flavonoid_library has 34 True values.
Column selleckchem_alkaloid_library has 45 True values.
Column selleckchem_np_library has 492 True values.
Column nencka_mce has 0 True values.
Column puretitre has 21 True values.
Column analyticon_megx_617 has 545 True values.
Column analyticon_natx_387 has 387 True values.


## Same workflow for split_inchikey

,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,selleckchem_np_library,selleckchem_phenol_library,selleckchem_terpenoid_library,selleckchem_organic_library,selleckchem_flavonoid_library,selleckchem_alkaloid_library,entries,split_inchikey,acquired,collaborators,accessible
nencka_mce,,,,,,,,,,,,,,,,,,,
True,39,2,1,0,0,6,6,6,6,1,0,3,2,1,5000,5000,5000,5000,5000


In [15]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["split_inchikey"]].copy()
  df = df.dropna(subset="split_inchikey").drop_duplicates(["split_inchikey"]).set_index(["split_inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)


merged_df

C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (139) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (7,51,55,57,58,60,61,62,64,65,67,69,75,76,82,87,90,99,157) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (71,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (44,51,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,entries
split_inchikey,,,,,,,,,
BGVLELSCIHASRV,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
SWMDAPWAQQTBOG,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
PZUSGRHVYDQLHR,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
FTSDLONCFCQDGA,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
KPFZCKDPBMGECB,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
DHOPKRHZTJAKOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1
PAVOYTTVGMVBBY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1
QQMRTEHNLOMJCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1


In [20]:
merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)

not_acquired = merged_df.loc[merged_df["acquired"] == False]
not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [23]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


Column targetmol_np_4320 has 1513 True values.
Column targetmol_np_3720 has 1096 True values.
Column selleckchem_subset has 10000 True values.


In [24]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")

Column targetmol_np_4320 has 1489 True values.
Column targetmol_np_3720 has 1077 True values.
Column selleckchem_subset has 10000 True values.


## Extracting unique structures in compound library

In [128]:
library_file_cleaned = r"C:\git\msn_library\data\compound_libraries\cleaned_tables\L6000-Targetmol-Natural Product Library for HTS-4320cpds_script_cleaned.tsv"
library_file = r"C:\git\msn_library\data\compound_libraries\raw_data\L6000-Targetmol-Natural Product Library for HTS-4320cpds_script.tsv"

In [129]:
cleaned_df = pu.read_dataframe(library_file_cleaned)
lib_df = pu.read_dataframe(library_file)

C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (71,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


In [130]:
target_mol = not_acquired.loc[not_acquired["targetmol_np_4320"] == True]
target_mol

,mce,nih,warth_xeno_poly,petras_np_library,koellensperger_mix,targetmol_np_4320,targetmol_np_3720,selleckchem_subset,entries,split_inchikey,acquired,collaborators,accessible
inchikey,,,,,,,,,,,,,
PROQIPRRNZUXQM-ZMSHIADSSA-N,NaN,NaN,True,NaN,NaN,True,True,NaN,3,PROQIPRRNZUXQM,False,True,True
FPQFYIAXQDXNOR-QDKLYSGJSA-N,NaN,NaN,True,NaN,NaN,True,True,NaN,3,FPQFYIAXQDXNOR,False,True,True
PXKLMJQFEQBVLD-UHFFFAOYSA-N,NaN,NaN,True,NaN,NaN,True,NaN,NaN,2,PXKLMJQFEQBVLD,False,True,True
LPEPZZAVFJPLNZ-SFHVURJKSA-N,NaN,NaN,True,NaN,NaN,True,True,NaN,3,LPEPZZAVFJPLNZ,False,True,True
YZBOVSFWWNVKRJ-UHFFFAOYSA-N,NaN,NaN,True,NaN,NaN,True,True,NaN,3,YZBOVSFWWNVKRJ,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CMZFNIMQBCBHEX-JRIAKSEUSA-N,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1,CMZFNIMQBCBHEX,False,False,False
OXEZOWCIRUNPIN-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1,OXEZOWCIRUNPIN,False,False,False
WRPAFPPCKSYACJ-UYQGGQRHSA-N,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1,WRPAFPPCKSYACJ,False,False,False


In [131]:
inchi_set = set(target_mol.index)

In [132]:
cleaned_df["new"] = [inchikey in inchi_set for inchikey in cleaned_df["inchikey"]]
cleaned_df = cleaned_df.loc[cleaned_df["new"] == True].drop_duplicates(subset="inchikey")
cleaned_df

,Bioactivity,Formula,ID,MolWt,Pathways,Receptor,Reference,Selected Plant,Source,Target,...,structure_source,synonyms,topical,unichem_id,unichem_url,unii,usan_stem_definition,withdrawn,zinc_id,new
0,"Lariciresinol is an enterolignan precursor, it...",C20H24O6,TN1853,360.40,Angiogenesis; Microbiology/virology; Tyrosine ...,Antifection; VEGFR,Antifungal activity of lariciresinol derived f...,Phyllanthus niruri,Plant,Antifection; VEGFR,...,parent_pubchem_cid,Lariciresinol;(+)-Lariciresinol;27003-73-2;NSC...,False,218240.0,https://www.ebi.ac.uk/unichem/compoundsources?...,73XCE5OZB0,NaN,False,ZINC000004098820,True
1,Prosaikogenin D is a natural product extracted...,C36H58O8,TN2335,618.84,Others,Others,"Lu Shi, et al. A Potential Anti-cancer Compoun...",Bupleurum chinensis DC.,Plant,Others,...,parent_pubchem_cid,Prosaikogenin D;Prosapogenin D;103629-72-7;HY-...,NaN,186850562.0,https://www.ebi.ac.uk/unichem/compoundsources?...,NaN,NaN,NaN,NaN,True
2,Cyclosporin C is a fungal metabolite against f...,C62H111N11O13,T19971,1218.61,Microbiology/virology,Antifungal;Antiviral,"Freitas-Silva O, et al. Tracing fungi secondar...",NaN,Microorganism,Antifungal;Antiviral,...,parent_pubchem_cid,Cyclosporin C;59787-61-0;7-Threonine-cyclospor...,False,66757849.0,https://www.ebi.ac.uk/unichem/compoundsources?...,6S744L5508,NaN,False,NaN,True
3,Licoricesaponin H2 is a natural product,C42H62O16,TN2289,822.92,Others,Others,Antimutagenic components in Glycyrrhiza agains...,Glycyrrhiza glabra L.,Plant,Others,...,parent_pubchem_cid,Licorice-saponin H2;Licorice saponin H2;118441...,NaN,96950573.0,https://www.ebi.ac.uk/unichem/compoundsources?...,6FO62043WK,NaN,NaN,ZINC000255227479,True
4,Steviol-​19-​O-​glucoside is a natural product...,C26H40O8,TN2237,480.59,Others,Others,"Li W., et al Development of Photoaffinity Prob...",Stevia rebaudiana,Plant,Others,...,parent_pubchem_cid,1185737-16-9;Steviol-​19-​O-​glucoside,NaN,48105924.0,https://www.ebi.ac.uk/unichem/compoundsources?...,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314,6-Ethoxychelerythrine has anti-bacterial activ...,C23H23NO5,T2S0058,393.44,Microbiology/virology,Antibacterial,"Rodríguez-Guzmán R, et al. Chemical constituen...",Chelidonium majus,Plant,Antibacterial,...,parent_pubchem_cid,SCHEMBL13103363;FT-0723636;79559-55-0;6-Ethoxy...,NaN,28777258.0,https://www.ebi.ac.uk/unichem/compoundsources?...,NaN,NaN,NaN,NaN,True
4315,Mogroside IV-E is a triterpenoid glycoside iso...,C54H92O24,T8300,1125.30,oxidation-reduction,Antioxidant,"Itkin M, et al. The biosynthetic pathway of th...",Siraitia grosvenorii Swingle,Plant,Antioxidant,...,parent_pubchem_cid,Mogroside IV-E;Mogroside IV;88915-64-4;HY-N245...,NaN,163385147.0,https://www.ebi.ac.uk/unichem/compoundsources?...,NaN,NaN,NaN,NaN,True
4316,Anisodamine Hydrobromide is an anticholinergic...,C17H24BrNO4,T8304,386.28,GPCR/G Protein; Neuroscience,mAChR; α1-adrenergic receptor,"Zhang WW, et al. Differential Neuropsychopharm...",Atropa belladonna L.,Plant,AChR; Adrenergic Receptor,...,parent_pubchem_cid,"anisodamine;55869-99-3;Hyoscyamine, 6-hydroxy-...",False,1075771.0,https://www.ebi.ac.uk/unichem/compoundsources?...,01343Q8EL8,NaN,False,ZINC000003197739,True
4318,Lipopolysaccharide is a highly immunogenic ant...,C205H366N3O117P5,T11855,4899.92,Others,Others,"Rietschel ET, et al. Bacterial endotoxin: mole...",NaN,microorganism,Others,...,parent_pubchem_cid,LPSLPS;NCGC00188939-01;T11855;Lipopolysaccharides,NaN,58462902.0,https://www.ebi.ac.uk/unichem/compoundsources?...,NaN,NaN,NaN,NaN,True


In [133]:
duplicates = cleaned_df[cleaned_df.duplicated(subset=["split_inchikey"], keep=False)]
duplicates[["ID", "input_name", "compound_name", "inchikey", "split_inchikey"]]

,ID,input_name,compound_name,inchikey,split_inchikey
5,TN1983,Neoisoastilbin,Neoisoastilbin,ZROGCCBNZBKLEL-XEXNSLJOSA-N,ZROGCCBNZBKLEL
6,T7030,anemarsaponin B,Anemarsaponin B,ROHLIYKWVMBBFX-UHFFFAOYSA-N,ROHLIYKWVMBBFX
11,TN7109,Prosaikogenin G,Prosaikogenin G,WSSVJIGMYVWUJL-LRSKSROMSA-N,WSSVJIGMYVWUJL
41,T2S0968,Sennoside D,Sennoside D,ZFWOUNNKSHIAFK-MIIKWYNKSA-N,ZFWOUNNKSHIAFK
51,TN1383,Timosaponin B III,Timosaponin B III,ROHLIYKWVMBBFX-XNZAAYBPSA-N,ROHLIYKWVMBBFX
...,...,...,...,...,...
4044,T0581,Isomaltose,ISOMALTOSE,DLRVVLDZNNYCBX-RTPHMHGBSA-N,DLRVVLDZNNYCBX
4126,TN1459,Bufarenogin,Bufarenogin,SOGONHOGEFLVPE-PUVOGLICSA-N,SOGONHOGEFLVPE
4134,TN7121,d-Epigalbacin,d-epigalbacin,QFUXQRHAJWXPGP-HKKFXGGESA-N,QFUXQRHAJWXPGP
4180,TN2107,Protoescigenin,Protoescigenin,VKJLHZZPVLQJKG-JAGYOTNFSA-N,VKJLHZZPVLQJKG


In [134]:
id_set = set(cleaned_df["ID"])

In [135]:
lib_df["new"] = [id in id_set for id in lib_df["ID"]]
lib_df = lib_df.loc[lib_df["new"] == True].drop(columns="new")
lib_df

,ID,compound_name,cas,Source,Selected Plant,category,smiles,Formula,MolWt,Pathways,Target,Receptor,Bioactivity,Reference
0,TN1853,Lariciresinol,27003-73-2,Plant,Phyllanthus niruri,Lignans,COc(cc(C[C@@H]1[C@H](CO)[C@@H](c(cc2)cc(OC)c2O...,C20H24O6,360.40,Angiogenesis; Microbiology/virology; Tyrosine ...,Antifection; VEGFR,Antifection; VEGFR,"Lariciresinol is an enterolignan precursor, it...",Antifungal activity of lariciresinol derived f...
1,TN2335,Prosaikogenin D,103629-72-7,Plant,Bupleurum chinensis DC.,Triterpenoids,C[C@@]([C@@]1([H])C=CC2=C3CC(C)(CC[C@]3(CO)[C@...,C36H58O8,618.84,Others,Others,Others,Prosaikogenin D is a natural product extracted...,"Lu Shi, et al. A Potential Anti-cancer Compoun..."
2,T19971,Cyclosporin C,59787-61-0,Microorganism,NaN,NaN,[C@H]([C@@H](C/C=C/C)C)(O)[C@@]1(N(C)C(=O)[C@H...,C62H111N11O13,1218.61,Microbiology/virology,Antifungal;Antiviral,Antifungal;Antiviral,Cyclosporin C is a fungal metabolite against f...,"Freitas-Silva O, et al. Tracing fungi secondar..."
3,TN2289,Licorice-saponin H2,118441-85-3,Plant,Glycyrrhiza glabra L.,Terpenoids,[H][C@@]12C[C@](C)(CC[C@]1(C)CC[C@]1(C)C2=CC(=...,C42H62O16,822.92,Others,Others,Others,Licoricesaponin H2 is a natural product,Antimutagenic components in Glycyrrhiza agains...
4,TN2237,Steviol-​19-​O-​glucoside,1185737-16-9,Plant,Stevia rebaudiana,Diterpenoids,C=C([C@@](CC1)(O)C2)C[C@@]32CC[C@]4([H])[C@@](...,C26H40O8,480.59,Others,Others,Others,Steviol-​19-​O-​glucoside is a natural product...,"Li W., et al Development of Photoaffinity Prob..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314,T2S0058,6-Ethoxychelerythrine,79559-55-0,Plant,Chelidonium majus,Alkaloids,O(C=1C=CC2=C3C=C(OCC)C=4C=C5OCOC5=CC4C3=[N+](C...,C23H23NO5,393.44,Microbiology/virology,Antibacterial,Antibacterial,6-Ethoxychelerythrine has anti-bacterial activ...,"Rodríguez-Guzmán R, et al. Chemical constituen..."
4315,T8300,Mogroside IV-E,88915-64-4,Plant,Siraitia grosvenorii Swingle,Terpenoids,C[C@H](CC[C@H](C(C)(C)O)O[C@H]1[C@@H]([C@H]([C...,C54H92O24,1125.30,oxidation-reduction,Antioxidant,Antioxidant,Mogroside IV-E is a triterpenoid glycoside iso...,"Itkin M, et al. The biosynthetic pathway of th..."
4316,T8304,Anisodamine Hydrobromide,55449-49-5,Plant,Atropa belladonna L.,Alkaloids,Br.[H][C@@]12C[C@H](O)[C@@]([H])(C[C@H](C1)OC(...,C17H24BrNO4,386.28,GPCR/G Protein; Neuroscience,AChR; Adrenergic Receptor,mAChR; α1-adrenergic receptor,Anisodamine Hydrobromide is an anticholinergic...,"Zhang WW, et al. Differential Neuropsychopharm..."
4318,T11855,Lipopolysaccharides,T11855,microorganism,NaN,Others,CCCCCCCCCCCCCCCC1[C@H](OC([C@H]([C@@H]1OC(=O)C...,C205H366N3O117P5,4899.92,Others,Others,Others,Lipopolysaccharide is a highly immunogenic ant...,"Rietschel ET, et al. Bacterial endotoxin: mole..."


In [136]:
pu.save_dataframe(lib_df, r"C:\git\msn_library\data\compound_libraries\raw_data\L6000-Targetmol-Natural Product Library for HTS-4320cpds_subset_new.tsv" )

In [118]:
duplicates = lib_df[lib_df.duplicated(subset=["ID"], keep=False)]
duplicates

,ID,compound_name,cas,Source,Selected Plant,category,smiles,Formula,MolWt,Pathways,Target,Receptor,Bioactivity,Reference


In [66]:
duplicates[["ID", "input_name", "compound_name", "inchikey", "split_inchikey"]]

,ID,input_name,compound_name,inchikey,split_inchikey
